# Resource estimates for 22 O tapered circuits

In [1]:
import pennylane as qml
from pennylane import numpy as np

import ionizer
from ionizer.transforms import ionize, commute_through_ms_gates, single_qubit_fusion_gpi

In [2]:
dev = qml.device("default.qubit", wires=range(5))

def exp_decomposition(param, y_wire, y_wire_sign=1):
    """Applies a compact version of Exp("Z..ZYZ..Z") for some parameter, 
    where we pass the index of the Y wire."""
    qml.RZ(np.pi/2, wires=y_wire)
    qml.RY(np.pi/2, wires=y_wire)
    qml.RZ(-np.pi/2, wires=y_wire)
    qml.CNOT(wires=[4, 3])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[3, 1])
    qml.CNOT(wires=[1, 0])
    qml.RZ(param, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[3, 1])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[4, 3])
    qml.RZ(y_wire_sign * param, wires=y_wire)
    qml.RZ(-np.pi/2, wires=y_wire)
    qml.RY(np.pi/2, wires=y_wire)
    qml.RZ(np.pi/2, wires=y_wire) 

@qml.qfunc_transform
def expand_rot_and_remove_zeros(tape):
    for op in tape:
        if op.name == "Rot":
            if not np.isclose(op.data[0], 0.0):
                qml.RZ(op.data[0], wires=op.wires)
            if not np.isclose(op.data[1], 0.0):
                qml.RY(op.data[1], wires=op.wires)
            if not np.isclose(op.data[2], 0.0):
                qml.RZ(op.data[2], wires=op.wires)
        else:
            qml.apply(op)

@qml.qnode(dev)
@expand_rot_and_remove_zeros
@qml.transforms.single_qubit_fusion()
def tapered_circuit_simplified(params):
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    qml.PauliX(wires=2)
            
    qml.RY(params[0], wires=3)
    qml.CNOT(wires=[3, 0])
    
    qml.SingleExcitation.compute_decomposition(params[1], wires=[0, 4])
    
    exp_decomposition(params[2], y_wire=0)
        
    qml.SingleExcitation.compute_decomposition(params[3], wires=[1, 3])
    qml.SingleExcitation.compute_decomposition(params[4], wires=[1, 4])

    exp_decomposition(params[5], y_wire=1)
    
    qml.SingleExcitation.compute_decomposition(params[6], wires=[2, 3])
    qml.SingleExcitation.compute_decomposition(params[7], wires=[2, 4])
    
    exp_decomposition(params[8], y_wire=2)
    return qml.probs()

In [3]:
params = np.array([ 0.96024933, -0.13428186,  0.03586267,  1.09763415,  0.09728663,
        -0.05481184, -1.30243657, -0.0782612 ,  0.06417085], requires_grad=True)

In [4]:
print(qml.draw(tapered_circuit_simplified)(params))

0: ──RZ(1.57)──RY(3.14)──RZ(-1.57)─╭X──RZ(2.36)──RY(1.57)──RZ(1.57)─╭X──RZ(0.07)──╭X──RZ(-3.14)
1: ──RZ(1.57)──RY(3.14)──RZ(-1.57)─│────────────────────────────────│─────────────│──╭X────────
2: ──RZ(1.57)──RY(3.14)──RZ(-1.57)─│────────────────────────────────│─────────────│──╰●────────
3: ──RY(0.96)──────────────────────╰●───────────────────────────────│─────────────│────────────
4: ──RZ(0.79)──RY(1.57)─────────────────────────────────────────────╰●──RY(-0.07)─╰●──RZ(3.14)─

───RY(0.79)─────────────────╭X──RZ(0.04)─╭X──RZ(-1.53)──RY(1.57)──RZ(1.57)───────────────────────
─────────────────────────╭X─╰●───────────╰●─╭X─────────╭X─────────RZ(2.36)──RY(1.57)──RZ(1.57)─╭X
─────────────────────────│──────────────────│──────────╰●──────────────────────────────────────│─
──────────────────────╭X─╰●─────────────────╰●─────────╭X─────────RZ(0.79)──RY(1.57)───────────╰●
───RY(1.57)──RZ(2.36)─╰●───────────────────────────────╰●─────────RZ(0.79)──RY(1.57)─────────────

────────────────────────────

## Transpile to Qiskit

In [5]:
from qiskit import QuantumCircuit
from qiskit import transpile

# Get the tape and run it through the tranaspiler; use the optimal
# params but convert to non-diff version
original_tape = tapered_circuit_simplified.qtape
original_qasm = original_tape.to_openqasm()

In [6]:
qiskit_circuit = QuantumCircuit.from_qasm_str(original_qasm)

transpiled_circuit = transpile(
    qiskit_circuit, 
    basis_gates=['rz', 'ry', 'cx', 'measure'],
    optimization_level=3
)                                 

In [7]:
# Convert back to a PennyLane function
new_qfunc = qml.from_qiskit(transpiled_circuit)

@qml.qnode(dev)
def tapered_circuit_transpiled():
    new_qfunc()
    # The Qiskit transpiler eats terminal RZs; we actually need
    # them when we measure the expval of a Hamiltonian, so add them back 
    qml.RZ(np.pi/2, wires=2)
    return qml.probs()

In [8]:
qml.specs(tapered_circuit_transpiled)()

/home/olivia/.conda/envs/ionizer/lib/python3.11/site-packages/pennylane_qiskit/converter.py:227: UserWarning: pennylane_qiskit.converter: The Measure instruction is not supported by PennyLane, and has not been added to the template.
  warnings.warn(


{'resources': Resources(num_wires=5, num_gates=94, gate_types=defaultdict(<class 'int'>, {'RY': 28, 'RZ': 31, 'CNOT': 35}), gate_sizes=defaultdict(<class 'int'>, {1: 59, 2: 35}), depth=55, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 59, 2: 35}),
 'gate_types': defaultdict(int, {'RY': 28, 'RZ': 31, 'CNOT': 35}),
 'num_operations': 94,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 5,
 'num_trainable_params': 0,
 'depth': 55,
 'num_device_wires': 5,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

## Transpile to SC device

In [36]:
from qiskit.transpiler import CouplingMap

coupling_map = CouplingMap([
    (0, 1), (1, 0), 
    (1, 2), (2, 1), 
    (1, 3), (3, 1), 
    (3, 5), (5, 3),
    (4, 5), (5, 4),
    (5, 6), (6, 5)
])

sc_transpiled_circuit = transpile(
    qiskit_circuit, 
    coupling_map=coupling_map,
    layout_method="sabre",
    routing_method="sabre",
    basis_gates=['rz', 'ry', 'cx', 'measure'],
)                                 

In [37]:
sc_transpiled_circuit.count_ops()

OrderedDict([('cx', 57),
             ('rz', 36),
             ('ry', 28),
             ('measure', 5),
             ('barrier', 1)])

In [38]:
sc_transpiled_circuit.depth()

90

## Transpile to trapped ion gates

In [12]:
@qml.qnode(dev)
@single_qubit_fusion_gpi
@ionizer.transforms.commute_through_ms_gates(direction="right")
@single_qubit_fusion_gpi
@ionizer.transforms.commute_through_ms_gates(direction="right")
@ionize
def tapered_circuit_ionized(params):
    new_qfunc()
    qml.RZ(np.pi/2, wires=2)
    return qml.probs()

In [13]:
print(qml.draw(tapered_circuit_ionized)(params))

0: ──────────────────────────────────────╭MS──GPI2(-1.57)──GPI(0.39)──GPI2(0.79)─╭MS──GPI(-0.03)─
1: ──────────────────────────╭MS─────────│───────────────────────────────────────│───────────────
2: ──GPI2(-2.36)──GPI(-0.39)─╰MS─────────│───────────────────────────────────────│───────────────
3: ──GPI2(-1.57)──GPI(1.09)───GPI2(2.18)─╰MS──GPI2(-1.57)────────────────────────│───────────────
4: ──GPI2(-2.36)──GPI(0.39)───GPI2(1.57)─────────────────────────────────────────╰MS──GPI2(-1.57)

──────────────────────────╭MS──GPI2(-1.09)──GPI(-1.96)──GPI2(0.30)─────────────────────────────╭MS
──────────────────────────│────────────────────────────────────────────────────╭MS──GPI2(1.57)─╰MS
──────────────────────────│────────────────────────────────────────────────────│──────────────────
──────────────────────────│────────────────╭MS──────────GPI2(3.14)──GPI2(1.57)─╰MS────────────────
───GPI(2.39)──GPI2(-1.50)─╰MS──GPI2(-1.57)─╰MS────────────────────────────────────────────────────

───GPI(-0.02)

In [14]:
qml.specs(tapered_circuit_ionized)(params)

{'resources': Resources(num_wires=5, num_gates=132, gate_types=defaultdict(<class 'int'>, {'GPI2': 59, 'GPI': 38, 'MS': 35}), gate_sizes=defaultdict(<class 'int'>, {1: 97, 2: 35}), depth=83, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 97, 2: 35}),
 'gate_types': defaultdict(int, {'GPI2': 59, 'GPI': 38, 'MS': 35}),
 'num_operations': 132,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 5,
 'num_trainable_params': 0,
 'depth': 83,
 'num_device_wires': 5,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}